In [5]:
!pip install -q tensorflow
import tensorflow as tf 
from tensorflow.keras import datasets, layers, models

In [6]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

11493376/11490434 [==============================] - 1s 0us/step


In [ ]:
# cnn 과 같은 이미지 데이터를 다룰 때, 입력 데이터로 4차원 텐서를 다룹니다. 
# 4 차원 데이터는 (image 수, channel 수, height, width )와 같은 구조를 갖습니다. 

# 머신러닝에서 데이터의 형태는 이와 같이 분류합니다. 
# - 스칼라 0 차원 텐서 
# - 배열 1 차원 텐서
# - 행렬 2 차원 텐서 
# - 텐서 N 차원 텐서 

# np.transpose 를 사용, axis 값을 통하여 학습 데이터의 포멧 변경가능하다. 


In [14]:
train_images.shape

(60000, 28, 28, 1)

In [ ]:
# CNN은 배치(batch) 크기를 제외하고 (이미지 높이, 이미지 너비, 컬러 채널) 크기의 텐서(tensor)를 입력으로 받습니다. 
# MNIST 데이터는 (흑백 이미지이기 때문에) 컬러 채널(channel)이 하나지만 컬러 이미지는 (R,G,B) 세 개의 채널을 가집니다. 
# 이 예에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하는 CNN을 정의하겠습니다. 
# 이 값을 첫 번째 층의 input_shape 매개변수로 전달합니다.
# Conv2D , MaxPooling2D 층을 쌓는 일반적인 패턴으로 합성곱 층을 정의 

# Conv2D (컨볼루션 필터의 수, (컨볼루션 커널의 행, 열), padding= 경계처리 방법, input_shape= 입력 형태를 정의, <행, 열, 채널 수>, activation = 활서오하 함수)
# Max Pooling 2D (pool_size = (수직, 수평 축소 비율 ) / if (2,2) => 출력 영상 크기는 입력 영상 크기의 반으로 줄어든다. 

In [11]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
# 케라스는 순차형(sequential) 모델 / 함수형 API(functional API) 두가지 유형이 있다. 

# 1. 순차형 - 차례로 계층을 쌓아 나가는 구조 
# 순차형 모델, .compile() 메서드를 사용하여 학습에 대한 설정을 해줘야 한다. 
# 필요한 인자 
# a. optimizer : 최적화 함수를 설정하는 부분, 
# b. loss : 손실함수를 설정해주는 부분 
# c. metrics : 모델의 성능을 판정하는데 사용하는 지표 함수 


In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# 모델을 완성하기 위해 마지막 합성곱 층의 출력 텐서 (4,4,64) 를 하나 이상의 dense 층에 주입하여 분류를 수행. 
# dense 층은 벡터 1D 를 입력으로 받는데, 현재 출력은 3D 텐서이다. 
# 3D 를 1D 로 펼치고, 하나 이상의 Dense 층을 그 위에 추가. 

In [17]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [18]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 24s 12ms/step - loss: 0.1386 - accuracy: 0.9586
Epoch 2/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0457 - accuracy: 0.9857
Epoch 3/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0312 - accuracy: 0.9901
Epoch 4/5
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0246 - accuracy: 0.9925
Epoch 5/5
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0181 - accuracy: 0.9945


In [19]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

313/313 - 1s - loss: 0.0327 - accuracy: 0.9905


In [20]:
print(test_acc)

0.9904999732971191
